In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,46008
2,Huelva,Confirmados PDIA 14 días,308
3,Huelva,Tasa PDIA 14 días,"60,019096985404445"
4,Huelva,Confirmados PDIA 7 días,163
5,Huelva,Tasa PDIA 7 dias,"31,763353274743263"
6,Huelva,Total Confirmados,46211
7,Huelva,Curados,45023
8,Huelva,Fallecidos,418


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  46008.0


/tmp/ipykernel_56717/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12649.0


/tmp/ipykernel_56717/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_56717/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_56717/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_56717/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 4230 personas en los últimos 7 días 

Un positivo PCR cada 2397 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,46008.0,163.0,308.0,513170.0,31.763353,60.019097,18.0
Huelva-Costa,27215.0,95.0,179.0,289548.0,32.809759,61.820493,12.0
Cartaya,2095.0,27.0,59.0,20083.0,134.442065,293.780810,7.0
Condado-Campiña,14289.0,40.0,83.0,156231.0,25.603113,53.126460,4.0
Sierra de Huelva-Andévalo Central,4078.0,28.0,46.0,67391.0,41.548575,68.258373,3.0
Ayamonte,2103.0,9.0,15.0,21104.0,42.645944,71.076573,2.0
Moguer,2008.0,9.0,26.0,21867.0,41.157909,118.900627,2.0
Minas de Riotinto,182.0,2.0,4.0,3812.0,52.465897,104.931794,1.0
San Bartolomé de la Torre,380.0,2.0,2.0,3761.0,53.177346,53.177346,1.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Campofrío,45.0,18.0,23.0,713.0,2524.544180,3225.806452,0.0
Granado (El),44.0,4.0,4.0,532.0,751.879699,751.879699,1.0
Zalamea la Real,123.0,3.0,9.0,3054.0,98.231827,294.695481,0.0
Cartaya,2095.0,27.0,59.0,20083.0,134.442065,293.780810,7.0
Palos de la Frontera,1061.0,9.0,16.0,11742.0,76.647931,136.262988,1.0
Moguer,2008.0,9.0,26.0,21867.0,41.157909,118.900627,2.0
Minas de Riotinto,182.0,2.0,4.0,3812.0,52.465897,104.931794,1.0
Isla Cristina,3153.0,8.0,17.0,21393.0,37.395410,79.465246,1.0
Palma del Condado (La),1431.0,3.0,8.0,10801.0,27.775206,74.067216,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Punta Umbría,1277.0,1.0,4.0,15355.0,6.512537,26.050147,0.0,0.250000
Zalamea la Real,123.0,3.0,9.0,3054.0,98.231827,294.695481,0.0,0.333333
Aracena,673.0,1.0,3.0,8255.0,12.113870,36.341611,1.0,0.333333
Moguer,2008.0,9.0,26.0,21867.0,41.157909,118.900627,2.0,0.346154
Palma del Condado (La),1431.0,3.0,8.0,10801.0,27.775206,74.067216,0.0,0.375000
Bollullos Par del Condado,1296.0,2.0,5.0,14387.0,13.901439,34.753597,0.0,0.400000
Cartaya,2095.0,27.0,59.0,20083.0,134.442065,293.780810,7.0,0.457627
Isla Cristina,3153.0,8.0,17.0,21393.0,37.395410,79.465246,1.0,0.470588
Condado-Campiña,14289.0,40.0,83.0,156231.0,25.603113,53.126460,4.0,0.481928
